In [6]:
import notebookjs
import glob
from notebookjs import execute_js
import random
from sklearn.calibration import calibration_curve
import sklearn
import shap
import pandas as pd
import numpy as np
import sys
from sklearn.metrics import confusion_matrix
from interpret.glassbox import ExplainableBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# sys.path.insert(0, '../core/')
from pycalibrate.calibrate import Calibrate

/home/joaorulff/miniconda3/envs/calibrate_deploy/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## User Simulation

In [7]:
X,y = shap.datasets.adult(display=True)

le = sklearn.preprocessing.LabelEncoder()
X['Sex'] = le.fit_transform(X['Sex'])
X = pd.get_dummies(X, columns=["Workclass", "Marital Status", "Occupation", "Relationship", "Race", "Country"])

scaler = sklearn.preprocessing.StandardScaler()
train_col_sacle = X[['Age','Education-Num','Hours per week']]
train_scaler_col = scaler.fit_transform(train_col_sacle)
train_scaler_col = pd.DataFrame(train_scaler_col,columns=train_col_sacle.columns)
X['Age'] = train_scaler_col['Age']
X['Education-Num'] = train_scaler_col['Education-Num']
X['Hours per week'] = train_scaler_col['Hours per week']

X_train, X_valid, y_train, y_valid = sklearn.model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

# clf = MLPClassifier(random_state=42, max_iter=100).fit(X_train, y_train)

mlp = MLPClassifier(random_state=42, max_iter=100).fit(X_train, y_train)
y_preds_mlp = mlp.predict_proba(X_valid)

rf = RandomForestClassifier(random_state=42).fit(X_train, y_train)
y_preds_rf = rf.predict_proba(X_valid)

lr = LogisticRegression(random_state=42).fit(X_train, y_train)
y_preds_lr = lr.predict_proba(X_valid)


# OHE labels
ohe = sklearn.preprocessing.OneHotEncoder()

y_valid_labels = ohe.fit_transform(y_valid.reshape(-1,1)).toarray()

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [8]:
dataset = X_valid[['Age', 'Education-Num', 'Sex', 'Capital Gain', 'Capital Loss', 'Hours per week', 'Workclass_ ?']]
c = Calibrate(data=dataset)

## adding models
c.add_model(y_preds_mlp, y_valid_labels, 'MLPClassifier')
c.add_model(y_preds_rf, y_valid_labels, 'RandomForest')
c.add_model(y_preds_lr, y_valid_labels, 'LogisticReg')

## plotting
c.visualize()